# Examples

This notebook allows to recreate the figures presented in the usage scenario section.

## GeoNames City Dataset

Source: https://geonames.org (Retrieved May 1, 2024)

In [1]:
import pandas as pd
geonames = pd.read_parquet('https://paper.jupyter-scatter.dev/geonames.pq')
geonames.head(2)

,Name,Longitude,Latitude,Mercator X,Mercator Y,Population,Continent,Country
0,Powidz,17.91926,52.41362,1994762.875,6875261.5,1000,Europe,Poland
1,Janowiec,21.88940,51.32359,2436716.750,6678734.0,1000,Europe,Poland


In [2]:
import jscatter

scatter = jscatter.Scatter(
    data=geonames,
    x='Longitude',
    y='Latitude',
    color_by='Continent',
    width=600,
    height=600
)
scatter.show()

In [3]:
import matplotlib

scatter.opacity(0.5)
scatter.size(by='Population', map=(1, 8, 10), norm=matplotlib.colors.AsinhNorm())
scatter.color(by='Population', map='magma', norm=matplotlib.colors.LogNorm(), order='reverse')

In [4]:
scatter.legend(True)
scatter.axes(True, labels=True)
scatter.tooltip(True, properties=['color', 'Latitude', 'Country'], preview='Name')

In [5]:
scatter.xy(x='Mercator X', y='Mercator Y')

In [6]:
scatter.selection(geonames.query('Population > 10_000_000').index)

In [7]:
geonames.iloc[scatter.selection()]

,Name,Longitude,Latitude,Mercator X,Mercator Y,Population,Continent,Country
118851,Seoul,126.97840,37.56600,1.413517e+07,4.518296e+06,10349312,Asia,"Korea, Republic of"
118852,Dhaka,90.40744,23.71040,1.006411e+07,2.718158e+06,10356500,Asia,Bangladesh
118853,Moscow,37.61556,55.75222,4.187345e+06,7.509247e+06,10381222,Europe,Russian Federation
118854,Wuhan,114.26667,30.58333,1.272011e+07,3.578754e+06,10392693,Asia,China
118855,Delhi,77.23149,28.65195,8.597370e+06,3.331421e+06,10927986,Asia,India
118856,Tianjin,117.17667,39.14222,1.304405e+07,4.742064e+06,11090314,Asia,China
118857,Karachi,67.01040,24.86080,7.459564e+06,2.858657e+06,11624219,Asia,Pakistan
118858,Mexico City,-99.12766,19.42847,-1.103484e+07,2.205447e+06,12294193,North America,Mexico
118859,Sao Paulo,-46.63611,-23.54750,-5.191508e+06,-2.698365e+06,12400232,South America,Brazil
118860,Mumbai,72.88261,19.07283,8.113255e+06,2.163512e+06,12691836,Asia,India


## Fashion MNIST Embeddings

Source: Fashion-MNIST: a Novel Image Dataset for Benchmarking Machine Learning Algorithms. Han Xiao, Kashif Rasul, Roland Vollgraf. [doi:10.48550/arXiv.1708.07747](https://doi.org/10.48550/arXiv.1708.07747)

In [8]:
import pandas as pd
fashion_mnist = pd.read_parquet('https://paper.jupyter-scatter.dev/fashion-mnist-embeddings.pq')

In [9]:
import anywidget
import traitlets
import traittypes

class ImagesWidget(anywidget.AnyWidget):
    _esm = """
    const baseUrl = 'https://paper.jupyter-scatter.dev/fashion-mnist-images/';
    export function render({ model, el }) {
      const container = document.createElement('div');
      container.classList.add('images-container');
      el.appendChild(container);

      const grid = document.createElement('div');
      grid.classList.add('images-grid');
      container.appendChild(grid);

      function renderImages() {
        grid.textContent = '';
        
        model.get('images').forEach((image) => {
          const imgId = String(image).padStart(5, '0');
        
          const img = document.createElement('div');
          img.classList.add('images-fashion-mnist');
          img.style.backgroundImage = `url(${baseUrl}${imgId}.png)`;
        
          grid.appendChild(img);
        });
      }

      model.on("change:images", renderImages);
      
      renderImages();
    }
    """

    _css = """
    .images-container {
      position: absolute;
      inset: 0;
      overflow: auto;
      background: black;
    }
    
    .images-grid {
      display: grid;
      grid-template-columns: repeat(auto-fit, minmax(32px, 1fr));
      align-content: flex-start;
      gap: 8px;
    }
    
    .images-fashion-mnist {
      width: 32px;
      height: 32px;
      background-repeat: no-repeat;
      background-position: center;
    }
    """

    images = traittypes.Array(default_value=[]).tag(sync=True)

In [10]:
import ipywidgets
import jscatter

images = ImagesWidget()

scatter = jscatter.Scatter(
    data=fashion_mnist,
    x='umapX',
    y='umapY',
    color_by='class',
    background_color='black',
)

ipywidgets.link((scatter.widget, 'selection'), (images, 'images'))

ipywidgets.AppLayout(center=scatter.show(), right_sidebar=images)

AppLayout(children=(ImagesWidget(layout=Layout(grid_area='right-sidebar')), HBox(children=(VBox(children=(Butt…

In [11]:
config = dict(
    data=fashion_mnist,
    background_color='#111111',
    color_by='class',
    legend=True,
    axes=False,
    zoom_on_selection=True,
)

pca = jscatter.Scatter(x='pcaX', y='pcaY', **config)
tsne = jscatter.Scatter(x='tsneX', y='tsneY', **config)
umap = jscatter.Scatter(x='umapX', y='umapY', **config)
cae = jscatter.Scatter(x='caeX', y='caeY', **config)

jscatter.compose(
    [(pca, "PCA"), (tsne, "t-SNE"), (umap, "UMAP"), (cae, "CAE")],
    sync_selection=True,
    sync_hover=True,
    rows=2,
)

GridBox(children=(VBox(children=(HTML(value='<b style="display: flex; justify-content: center; margin: 0 0 0 3…

## Market Capitelization

Source: https://ycharts.com (Retrieved June 2, 2024)

In [12]:
import pandas as pd
market_cap = pd.read_parquet('https://paper.jupyter-scatter.dev/market-capitalization.pq')
market_cap.head(2)

,Symbol,Name,Sector,Industry,Date,Market Capitalization
0,MSFT,Microsoft Corporation,Technology,Software - Infrastructure,1559260800000,947737317
1,MSFT,Microsoft Corporation,Technology,Software - Infrastructure,1559520000000,918312097


In [13]:
import jscatter

scatter = jscatter.Scatter(data=market_cap, x='Date', x_scale='time', y='Market Capitalization')
scatter.axes(grid=True)
scatter.legend(True)
scatter.color(by='Sector')
scatter.connect(by='Symbol')
scatter.connection_color('inherit')
scatter.connection_size(3)
scatter.connection_opacity(0.5)
scatter.tooltip(True, preview='Name', properties=['x', 'y', 'color', 'Industry'])
scatter.height(480)
scatter.show()